In [26]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import googlemaps
import gmaps
import os
import requests
import shutil
# from IPython.display import Image

In [27]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [28]:
input_folder = '../data/railways/swissrail/'
input_file = input_folder+'netw_1.shp'

output_folder = '../data/output_images/Switzerland/'
output_csv = output_folder+'SWZ.csv'

update_folder = output_folder+'set_3/'

#### Import routes

In [29]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4326'})

routes.head()

,OBJECTID,xtf_id,Datenherr_,Datenher_1,Nummer,Name,BeginnGuel,EndeGuelti,Bearbeitun,Stand,SHAPE_Leng,geometry
0,1.0,ch14uvag00086875,1,SBB CFF FFS,702,Turgi - Koblenz - Waldshut,2014/09/03 00:00:00.000,None,2017/12/12 00:00:00.000,2017/12/10 00:00:00.000,15252.174997,LINESTRING (8.253483954124018 47.4918094079282...
1,2.0,ch14uvag00086926,1,SBB CFF FFS,824,Romanshorn - Konstanz,2014/09/03 00:00:00.000,None,2017/12/12 00:00:00.000,2017/12/10 00:00:00.000,18733.681101,LINESTRING (9.379483396212201 47.5655289670994...
2,3.0,ch14uvag00090328,122,BLT,L004,Reinach BL Süd - Reinach BL Süd,2014/10/01 00:00:00.000,None,2017/12/12 00:00:00.000,2017/12/10 00:00:00.000,118.817126,LINESTRING (7.591947809955224 47.4851590292389...
3,4.0,ch14uvag00086829,1,SBB CFF FFS,540,Olten - Wöschnau,2014/09/03 00:00:00.000,None,2017/12/12 00:00:00.000,2017/12/10 00:00:00.000,11579.008319,LINESTRING (7.90783042339883 47.35192256271322...
4,5.0,ch14uvag00067525,183,WSB,644,Aarau - Menziken,2009/01/01 00:00:00.000,None,2017/12/12 00:00:00.000,2017/12/10 00:00:00.000,22749.783132,LINESTRING (8.052739943168765 47.3908523444561...


In [5]:
routes.shape

(454, 12)

#### Get points from routes

In [30]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

457696 points


In [31]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
# curr_df['Catenary'] = curr_df['Catenary'].astype(int)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,9.379483396212201_47.565528967099404,9.379483,47.565529,1
1,9.367078340136002_47.57620856433524,9.367078,47.576209,1
2,9.347995664070325_47.583125270732666,9.347996,47.583125,1
3,9.32926653044524_47.59035302006366,9.329267,47.590353,1
4,9.311405490706957_47.598623942532434,9.311405,47.598624,1


In [34]:
'''
Get subselection of points excuding current points
'''
rows = curr_df.index.tolist()
curr_points = []
for row in rows:
    tmp = curr_df.iloc[row].Name
    space = tmp.find('_') 
    longitude = tmp[:space]
    latitude = tmp[space+1:]
    curr_points.append((longitude,latitude))

route_points = []
for point in range(0,len(points),4000):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

115


#### Create working dataset

In [35]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,8.253483954124018_47.49180940792827,8.253484,47.491809
1,7.971785341015192_47.35391228860773,7.971785,47.353912
2,8.688566454316728_46.787368312832825,8.688566,46.787368
3,8.850249554643572_46.45433988864974,8.850250,46.454340
4,9.487109536378044_47.21931994620723,9.487110,47.219320


In [36]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))

# Set map centerpoint
coords = marker_points[int((len(marker_points)/2))]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 7
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [39]:
'''
Get satellite preview for image
'''

row = 10

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [37]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [6]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 